# Importing Libraries

In [45]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm, trange

import tensorflow as tf
from tensorflow import keras
from keras import layers
import gc

import warnings
warnings.filterwarnings('ignore')

In [46]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "AVAILABLE" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.10.0
Keras Version: 2.10.0

GPU is AVAILABLE


In [47]:
#set tensorflow to user gpu if available
if gpu:
    print("Using GPU")
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
else:
    print("Not using CPU")

Using GPU


# Data Analysis

In [68]:
df = pd.read_csv('data/train_0.csv')
df.shape

(2149381, 61)

In [49]:
df.head()

,game_num,event_id,event_time,ball_pos_x,ball_pos_y,ball_pos_z,ball_vel_x,ball_vel_y,ball_vel_z,p0_pos_x,...,boost0_timer,boost1_timer,boost2_timer,boost3_timer,boost4_timer,boost5_timer,player_scoring_next,team_scoring_next,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,1,1002,-33.313030,-0.0,0.0,1.8548,-0.0,0.0,0.0,41.8048,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
1,1,1002,-33.206146,-0.0,0.0,1.8548,-0.0,0.0,0.0,42.2616,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
2,1,1002,-33.098114,-0.0,0.0,1.8548,-0.0,0.0,0.0,43.2270,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
3,1,1002,-32.993190,-0.0,0.0,1.8548,-0.0,0.0,0.0,43.8984,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
4,1,1002,-32.887756,-0.0,0.0,1.8548,-0.0,0.0,0.0,44.9606,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0


# Data Engineering

In [ ]:
goal1_pos_x = 10
goal1_pos_y = -105
goal1_pos_z = 0

goal2_pos_x = -10
goal2_pos_y = -105
goal2_pos_z = 0

In [ ]:
#calculate distance between ball and player
def distance(x1, y1, z1, x2, y2, z2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
    
#calculate velocity of ball
def velocity(x1, y1, z1):
    return np.sqrt(x1**2 + y1**2 + z1**2)
    
def get_normalized_df(df):
   df.fillna(0, inplace=True)
    
   for j in range(0, 6):
      df[f'p{j}_dis_ball'] = distance(df['ball_pos_x'], df['ball_pos_y'], df['ball_pos_z'], df[f'p{j}_pos_x'], df[f'p{j}_pos_y'], df[f'p{j}_pos_z'])
      df[f'p{j}_dis_goal1'] = distance(goal1_pos_x, goal1_pos_y, goal1_pos_z, df[f'p{j}_pos_x'], df[f'p{j}_pos_y'], df[f'p{j}_pos_z'])
      df[f'p{j}_dis_goal2'] = distance(goal2_pos_x, goal2_pos_y, goal2_pos_z, df[f'p{j}_pos_x'], df[f'p{j}_pos_y'], df[f'p{j}_pos_z'])
      df[f'p{j}_velocity'] = velocity(df[f'p{j}_vel_x'], df[f'p{j}_vel_y'], df[f'p{j}_vel_z'])

   df[f'ball_vel'] = velocity(df[f'ball_vel_x'], df[f'ball_vel_y'], df[f'ball_vel_z'])
   df[f'ball_dis_goal1'] = distance(goal1_pos_x, goal1_pos_y, goal1_pos_z, df[f'ball_pos_x'], df[f'ball_pos_y'], df[f'ball_pos_z'])
   df[f'ball_dis_goal2'] = distance(goal2_pos_x, goal2_pos_y, goal2_pos_z, df[f'ball_pos_x'], df[f'ball_pos_y'], df[f'ball_pos_z'])

   df[f't1_dis_ball'] = df[f'p0_dis_ball'] + df[f'p1_dis_ball'] + df[f'p2_dis_ball']
   df[f't1_dis_goal1'] = df[f'p0_dis_goal1'] + df[f'p1_dis_goal1'] + df[f'p2_dis_goal1']
   df[f't1_dis_goal2'] = df[f'p0_dis_goal2'] + df[f'p1_dis_goal2'] + df[f'p2_dis_goal2']

   df[f't2_dis_ball'] = df[f'p3_dis_ball'] + df[f'p4_dis_ball'] + df[f'p5_dis_ball']
   df[f't2_dis_goal1'] = df[f'p3_dis_goal1'] + df[f'p4_dis_goal1'] + df[f'p5_dis_goal1']
   df[f't2_dis_goal2'] = df[f'p3_dis_goal2'] + df[f'p4_dis_goal2'] + df[f'p5_dis_goal2']

   df = (df - df.mean()) / df.std()
    
   return df

In [ ]:
for i in range(10):
    path = f'data/train_{i}.csv'
    df_temp= pd.read_csv(path)

    #read 10 files into one dataframe
    if i == 0:
        df = df_temp.copy()
    else:
        df = pd.concat([df, df_temp], axis=0)

df.drop(['game_num', 'event_id', 'event_time', 'player_scoring_next', 'team_scoring_next'], axis=1, inplace=True)

# Data loading and preprocessing 
df['no_team_scored'] = np.logical_xor(df['team_A_scoring_within_10sec'], df['team_B_scoring_within_10sec'])
df['no_team_scored'] = (~df['no_team_scored']).astype(int)

df = get_normalized_df(df)

features = df.drop(['team_A_scoring_within_10sec', 'team_A_scoring_within_10sec', 'no_team_scored'], axis=1)
target = df.loc['team_A_scoring_within_10sec', 'team_A_scoring_within_10sec', 'no_team_scored']

# Model Development

In [51]:
input_ = keras.layers.Input(shape=[87])
hidden1 = keras.layers.Dense(128, activation='relu')(input_)
hidden2 = keras.layers.Dense(128, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(3, activation='softmax')(concat)
model = keras.Model(inputs=[input_], outputs=[output])

model.compile(optimizer='SGD', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [52]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 87)]         0           []                               
                                                                                                  
 dense_9 (Dense)                (None, 128)          11264       ['input_4[0][0]']                
                                                                                                  
 dense_10 (Dense)               (None, 128)          16512       ['dense_9[0][0]']                
                                                                                                  
 concatenate_3 (Concatenate)    (None, 215)          0           ['input_4[0][0]',                
                                                                  'dense_10[0][0]']         

In [56]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [58]:
from tabnanny import verbose

X_train, X_val,\
y_train, y_val = train_test_split(features, target,
                                    test_size = 0.01,
                                    random_state=0)

print(f'Training on dataset.')

checkpoint_cb = keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(X_train, y_train, 
            epochs=1, 
            steps_per_epoch=5000, 
            batch_size=32,
            validation_data=(X_val, y_val), 
            verbose=1,
            callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

del df, X_train, X_val, y_train, y_val
gc.collect()
print()

Training on dataset.
Epoch 1/100


2022-10-28 18:56:42.051688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4999/5000 [============================>.] - ETA: 0s - loss: 0.4491 - accuracy: 0.8884

2022-10-28 18:57:26.116312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5000/5000 [==============================] - 396s 79ms/step - loss: 0.4490 - accuracy: 0.8884 - val_loss: 66.7512 - val_accuracy: 0.0838
Epoch 2/100
5000/5000 [==============================] - 1227s 245ms/step - loss: 0.4291 - accuracy: 0.8877 - val_loss: 66.7609 - val_accuracy: 0.0838
Epoch 3/100
5000/5000 [==============================] - 378s 76ms/step - loss: 0.4247 - accuracy: 0.8893 - val_loss: 66.7631 - val_accuracy: 0.0837
Epoch 4/100
5000/5000 [==============================] - 377s 76ms/step - loss: 0.4267 - accuracy: 0.8886 - val_loss: 66.7578 - val_accuracy: 0.0838
Epoch 5/100
5000/5000 [==============================] - 377s 75ms/step - loss: 0.4241 - accuracy: 0.8895 - val_loss: 66.7639 - val_accuracy: 0.0837
Epoch 6/100
5000/5000 [==============================] - 378s 76ms/step - loss: 0.4201 - accuracy: 0.8909 - val_loss: 66.7305 - val_accuracy: 0.0838
Epoch 7/100
5000/5000 [==============================] - 377s 75ms/step - loss: 0.4252 - accuracy: 0.8891 - val_loss

## Predictions
One of the important factor here is predicting three classes B's score, A's Score and no ones score to get better results on the leaderboard. Due to this reason only sum of the probabilities for just A and B won't be equal to one.

In [72]:
df_test = pd.read_csv('data/test.csv')
df_test = df_test.drop(df_test.columns[0], axis=1)

(701143, 55)

In [61]:
df_test = get_normalized_df(df_test)
df_test.isnull().sum().sum()

0

In [75]:
preds = model.predict(df_test)

(701142, 54)

In [66]:
ss = pd.read_csv('data/sample_submission.csv')
ss['team_A_scoring_within_10sec'] = preds[:,0]
ss['team_B_scoring_within_10sec'] = preds[:,1]
ss.to_csv('Submission.csv', index=False)
ss.head()

,id,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,0,0.000000e+00,1.000000
1,1,8.624833e-16,0.987074
2,2,6.146056e-24,1.000000
3,3,0.000000e+00,1.000000
4,4,1.281038e-34,1.000000
